# Machine learning avec les modules Python tensorflow2/keras :

# Entraînement / exploitation d'un réseau de neurones dense pour la reconnaissance de chiffres manuscrits

version 2.0 du 4 avril 2022

<div class="alert alert-block alert-danger">
<span style="color:brown;font-family:arial;font-size:normal"> 
    Il est fondamental d'utiliser un <span style="font-weight:bold;">Environnement Virtuel Python</span> (EVP) pour chaque projet Python important : l'EVP permet de maîtriser pour chaque projet les versions de l'interpréteur Python et des modules "sensibles" (comme tensorflow par exemple).</span></div>

Les <i>notebooks</i> de l'APP-ML doivent être chargés dans un processus `jupyter notebook` lancé dans l'EVP <b><span style="color: rgb(200, 51, 102);">minfo_ml</span></b> créé en suivant la procédure du document `Consignes.pdf`. Ils doivent être travaillés dans l'ordre :

- `ML1_MNIST.ipynb` : vérifier le bon fonctionnement de l’EVP <b><span style="color: rgb(200, 51, 102);">minfo_ml</span></b>, charger et utiliser les données de la banque MNIST (images et labels).
- `ML2_DNN.ipynb` : construire un réseau dense, l’entraîner avec les données de la banque MNIST et afficher ses performances.
- `ML3_DNN_suite.ipynb` : charger un réseau dense entraîné et l’exploiter avec les données de test  MNIST.
- `ML4_CNN.ipynb` : construire un réseau convolutif, l’entraîner avec la banque MNIST, afficher ses performances et à l'exploiter avec les données de test.

**Le notebook `ML2_DNN.ipynb` doit être travaillé avant celui-ci.**

## Objectifs d'apprentissage visés :
- Savoir recharger la structure du réseau entraîné et ses poids
- Savoir exploiter le réseau entraîné rechargé avec la méthode `predict`
- Savoir afficher l amatrice de confusion.

## Import des modules Python
Le module **keras** qui permet une manipulation de haut niveau des objets **tensorflow** est intégré dans le module **tensorflow** (tf) depuis la version 2. <br>
La documentation du module **tf.keras** à consulter pour cet APP est ici : https://www.tensorflow.org/api_docs/python/tf/keras. 

In [ ]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import sys, cv2
import matplotlib.pyplot as plt
print(f"Python    : {sys.version.split()[0]}")
print(f"tensorflow: {tf.__version__} incluant keras {keras.__version__}")
print(f"numpy     : {np.__version__}")
print(f"OpenCV    : {cv2.__version__}")

Incrustation des tracés matplotlib dans le notebook et import de modules utiles :

In [ ]:
%matplotlib inline

# Rappel de la structure du réseau dense construit
- **couche d'entrée** : 784 valeurs comprises entre 0 et 1 (les pixels des image MNIST 28 $\times$ 28 mis sous forme d'un vecteur de 784 nombres `float` normalisés),
- **couche cachée** : 784 neurones utilisant la fonction d'activation `relu`,
- **couche de sortie** : 10 neurones, pour la classification des images en 10 classes associées aux chiffres {0,1,2...9}, utilisant la fonction d'activation `softmax` adaptée aux problèmes de classification.

<p style="text-align:center; font-style:italic; font-size:12px;">
    <img src="img/archiReseau.png" alt="archiReseau.png" style="width: 900px;"><br> 
    [crédit image : JLC]
</p>

## Travail à faire
### 1 - Charger et pré-traiter les données de test MNIST<br>2 - Recharger la structure du réseau entrainé et ses poids<br>3 - Exploiter le réseau entraîné : méthode predict<br>4 - Afficher la matrice de confusion


## 1 - Charger et pré-traiter les données de test MNIST

Le travail a été traité dans le *notebook* `ML1_MNIST.ipynb` :

In [ ]:
# Charger les données du MNIST :
(im_train, lab_train), (im_test, lab_test) = tf.keras.datasets.mnist.load_data()

# Définir les paramètres importants :
nb_im_test  = im_test.shape[0]     # nombre d'images de test 
nb_pixel    = im_test[0].size      # nombre d'éléments (des pixels) de la première image de test 
nb_classe   = len(set(lab_test))   # nombre de classe à identifier (les 10 chiffres de 0 à 9)

print(f"{nb_im_test} images de test")
print(f"{nb_pixel} pixels dans chaque image")
print(f"{nb_classe} classes à reconnaître (les chiffres de 0 à 9)")

# Mettre à plat et normaliser les matrices:
x_test  = im_test.reshape(nb_im_test, nb_pixel)/im_test.max()

# 'one-hot' encoding' des labels :
from tensorflow.keras.utils import to_categorical
y_test  = to_categorical(lab_test)

## 2 - Recharger la structure du réseau entrainé et ses poids

La  méthode `load_model` de la classe `tf.keras.models` permet de recharger **la structure et les poids** d'un réseau entraîné.<br>
De cette façon, on peut remettre le réseau de neurone dans son état initial avant de commencer un entraînement :

In [ ]:
import os

# définir le chemin d'accès du réseau entraîné dans le notebook précédent :
model_path = os.path.join('models', 'trained1_model')

# charger le réseau
model = tf.keras.models.load_model(model_path)

model.summary()

## 3 - Exploiter le réseau entraîné : méthode `predict`

La méthode `predict` de l'objet `model` permet de calculer les inférences du réseau pour une ou plusieurs entrées (voir la méthode `predict`dans la page [tf.keras.Sequential](https://www.tensorflow.org/api_docs/python/tf/keras/Sequential#predict)).

La cellule ci-dessous montre la mise en oeuvre de la méthode `predict` :

In [ ]:
i = 100  # numéro de l'image de test choisie

# afficher l'image :
from utils.tools import plot_images
plot_images(im_test,i,1,1) ; plt.show()

# inférence du réseau entrainé pour l'image choisie: 
rep = model.predict(x_test[i:i+1])      # Attention: x doit être un tableau de matrices...
                                        # => x[i] ne convient pas !

print(f"Inférence du réseau pour l'image n° {i} :\n{rep[0]}")

pour rendre plus lisible la sortie du réseau, on peut limiter à 2 chiffre après la virgule l'affichage du tableau numpy :

In [ ]:
with np.printoptions(formatter={'float':'{:.2f}'.format}):    
    print(f"Inférence du réseau pour l'image n° {i} arrondie à 2 chiffre :\n{rep[0]}")

La méthode `argmax` des tableaux *ndarray* de *numpy* permet d'obtenir le rang de la valeur maximale :

In [ ]:
print(f"Le label prédit par le réseau est rep[0].argmax() : {rep[0].argmax()}")
print(f"Le label associé à l'image choisie est lab_test[{i}] : {lab_test[i]}")

### Utilité de la méthode `argmax` de numpy pour décoder le tableau de vecteurs *one-hot* renvoyé par la méthode `predict`

Quand on calcule une inférence du réseau `model` avec les images du tableau `x_test` par exemple, on obtient un résultat qui est un tableau de vecteurs *one-hot* :

In [ ]:
results = model.predict(x_test)
print("forme du tableau 'results':", results.shape)
print("allure des vecteurs du tableau 'result' :")
with np.printoptions(formatter={'float':'{:.2f}'.format}): 
    print("results[0]  :", results[0])
    print("results[-1] :", results[-1])

Avec l'expression `results.argmax(axe=-1)`, on obtient le tableau des `argmax` de chaque vecteur $\leadsto$ c'est directement le tableau des chiffres reconnus par le réseau :

In [ ]:
inferences = results.argmax(axis=-1)
print(f"inferences.shape: {inferences.shape}, inferences.dtype: {inferences.dtype}")
print(f"contenu du tableau inferences : {inferences}")

On peut comparer le tableau `inférences` et le tableau `lab_test` avec l'operateur `==` (cela a un sens avec les tableaux *ndarray* du module *numpy*) :

In [ ]:
inferences == lab_test

en comptant le nombre de `True` dans le résultat, on a directement le nombre d'inférences justes :

In [ ]:
reponse_ok = (inferences == lab_test)
print(f"nombre de réponses justes : {reponse_ok.sum()} sur {nb_im_test} images de test")
precision = reponse_ok.sum()/nb_im_test*100
print(f"performance du réseau entraîné : {precision} % de réponses justes")

## 4 - Afficher la matrice de confusion

La fonction `show_cm` du module `utils.tools` affiche la **matrice de confusion** sui permet de visualiser :
- sur la diagonale : les bonnes réponses du réseau, avec dans chaque case le nombre de bonnes réponses
- hors diagonale : les erreurs du réseau, avec dans chaque case la fréquence d'apparition de l'erreur.

In [ ]:
from utils.tools import show_cm
help(show_cm)

Faire afficher la matrice de confusion avec la fonction `show_cm` en lui passant les argumets : le tableau `lab_test` des labels attendus, le tableau `results` des labels calculés par le model et la liste des classes :

### Suite :

Tu peux maintenant charger le notebook `ML4_CNN.ipynb` pour apprendre à construire et exploiter un réseau de neurones convolutif.

# Autres ressources intéressantes... des vidéos :

In [2]:
%%HTML
<iframe src="https://www.youtube.com/embed/trWrEWfhTVg" width="800" height="450" frameborder="0" allow="accelerometer; autoplay; encrypted-media; gyroscope; picture-in-picture" allowfullscreen></iframe>

In [3]:
%%HTML
<iframe src="https://www.youtube.com/embed/aircAruvnKk" width="800" height="450" frameborder="0" allow="accelerometer; autoplay; encrypted-media; gyroscope; picture-in-picture" allowfullscreen></iframe>

In [4]:
%%HTML
<iframe src="https://www.youtube.com/embed/IHZwWFHWa-w" width="800" height="450" frameborder="0" allow="accelerometer; autoplay; encrypted-media; gyroscope; picture-in-picture" allowfullscreen></iframe>

In [5]:
%%HTML
<iframe src="https://www.youtube.com/embed/Ilg3gGewQ5U" width="800" height="450" frameborder="0" allow="accelerometer; autoplay; encrypted-media; gyroscope; picture-in-picture" allowfullscreen></iframe>